In [1]:
import pandas as pd
import numpy as np
import random
import mysql.connector 
from mysql.connector import Error


In [2]:
from bokeh.embed import file_html,components
from bokeh.resources import CDN
from bokeh.io import output_notebook,show,output_file
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.models import Range1d,LinearAxis,ColumnDataSource,HoverTool

In [3]:
from bokeh.embed import components
from bokeh.resources import INLINE

In [4]:
try:
    connection=mysql.connector.connect(
        host='localhost',
        port=3306,
        user='root',
        password='89JQuery78#',
        db='papercompany')
        QUERY_CONSULTA="""SELECT sellers.NAME AS sellers,products.PRODUCTNAME,products.PRODUCTPRICE*sells.QUANTITY profit
                        FROM sells
                        INNER JOIN sellers ON sells.IDSELLER=sellers.IDSELLER
                        INNER JOIN products ON sells.IDPRODUCTS=products.IDPRODUCTS
                    """

    DATA_FRAME=pd.read_sql_query(QUERY_CONSULTA,connection)
    

except Exception as ex:
    raise Exception(ex)
finally:
    if connection.is_connected():
        connection.close()
        print('La coneccion se ha cerrado exitosamente')
    else:
        print('La conección la cerro pandas')

La coneccion se ha cerrado exitosamente


C:\Users\ferna\AppData\Local\Temp\ipykernel_11248\2995902192.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DATA_FRAME=pd.read_sql_query(QUERY_CONSULTA,connection)


In [5]:
#Realizando el pareto
PRODUCTS=DATA_FRAME.groupby(['sellers','PRODUCTNAME'])['profit'].sum()
PRODUCTS_U=PRODUCTS.unstack(level=1,fill_value=0)
PRODUCTS_U=PRODUCTS_U.reset_index()


In [6]:
PRODUCTS_TOTAL=DATA_FRAME.groupby(['sellers'])['profit'].sum().reset_index()


In [7]:
# UNION DE TABLAS
TABLA_TOTAL=pd.merge(PRODUCTS_U,PRODUCTS_TOTAL)
TABLA_TOTAL_SORT=TABLA_TOTAL.sort_values(by='profit',ascending=False)
TABLA_TOTAL_SORT

,sellers,Bond paper,Cardstock,Coasted paper,Copy paper,Fish paper,Inkjet paper,Kraft paper,Laid paper,Mette paper,Newsprint,Offset paper,Photo paper,Tracing paper,profit
4,Jacob Carrillo,0.0,400.0,630.0,0.0,210.0,1350.0,720.0,0.0,0.0,960.0,0.0,0.0,315.0,4585.0
15,Terrence Johnson,0.0,0.0,0.0,0.0,0.0,300.0,0.0,0.0,1950.0,0.0,0.0,0.0,1050.0,3300.0
8,Mikayla Johnson,550.0,100.0,315.0,2000.0,0.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0,3085.0
2,Chelsey Taylor,0.0,0.0,0.0,0.0,1995.0,0.0,1080.0,0.0,0.0,0.0,0.0,0.0,0.0,3075.0
7,Mary Goodman,0.0,0.0,0.0,1600.0,0.0,1350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2950.0
1,Brenda Garcia,0.0,0.0,0.0,0.0,1050.0,0.0,0.0,770.0,1050.0,0.0,0.0,0.0,0.0,2870.0
12,Robert Baker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,960.0,1000.0,0.0,0.0,1960.0
14,Ryan Cooper,0.0,0.0,0.0,0.0,945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,945.0,1890.0
17,Whitney Brown,0.0,600.0,0.0,0.0,1260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1860.0
11,Regina Gross,110.0,0.0,0.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,770.0,0.0,1680.0


# Definiendo la grafica a mostrar (Barras acumuladas)

In [8]:
sellers=list(TABLA_TOTAL_SORT['sellers'])
columns=list(TABLA_TOTAL_SORT.columns)

In [9]:
figure1=figure(x_range=sellers,height=250,title='Profit',plot_width=1300,plot_height=1200,
              tools='reset,box_zoom',toolbar_location='below',tooltips="$name @sellers:@$name")
figure1.xgrid.grid_line_color=None

figure1.xaxis.axis_label='Sellers'
figure1.yaxis.axis_label='$ Profit'
figure1.xaxis.major_label_text_font_size='50px'
figure1.xgrid.grid_line_color=None
figure1.ygrid.grid_line_alpha=0.7
figure1.yaxis.major_label_text_font_size='50px'

figure1.toolbar.autohide=True
figure1.y_range.start = 0
figure1.x_range.range_padding = 0.1
figure1.xaxis.major_label_orientation = 1
figure1.xgrid.grid_line_color = None


In [11]:
figure1.vbar_stack(columns[1:-1],x='sellers', width=0.9, color=['#922B21','#B03A2E','#76448A','#6C3483',
                                                              '#1F618D','#2874A6','#117864','#0B5345',
                                                             '#196F3D','#7D6608','#6E2C00','#F5CBA7',
                                                             '#85C1E9'],line_color="#17202A", 
                    source=TABLA_TOTAL_SORT,legend_label=columns[1:-1])


[GlyphRenderer(id='1477', ...),
 GlyphRenderer(id='1520', ...),
 GlyphRenderer(id='1565', ...),
 GlyphRenderer(id='1612', ...),
 GlyphRenderer(id='1661', ...),
 GlyphRenderer(id='1712', ...),
 GlyphRenderer(id='1765', ...),
 GlyphRenderer(id='1820', ...),
 GlyphRenderer(id='1877', ...),
 GlyphRenderer(id='1936', ...),
 GlyphRenderer(id='1997', ...),
 GlyphRenderer(id='2060', ...),
 GlyphRenderer(id='2125', ...)]

In [12]:
show(figure1)

In [14]:
def barra_stackeada(DATA_FRAME,column_index,column_columns,column_values):
    PRODUCTS=DATA_FRAME.groupby([column_index,column_columns])[column_values].sum()
    PRODUCTS_U=PRODUCTS.unstack(level=1,fill_value=0)
    PRODUCTS_U=PRODUCTS_U.reset_index()
    PRODUCTS_TOTAL=DATA_FRAME.groupby([column_index])[column_values].sum().reset_index()
    # UNION DE TABLAS
    TABLA_TOTAL=pd.merge(PRODUCTS_U,PRODUCTS_TOTAL)
    TABLA_TOTAL_SORT=TABLA_TOTAL.sort_values(by=column_values,ascending=False)
    sellers=list(TABLA_TOTAL_SORT[column_index])
    columns=list(TABLA_TOTAL_SORT.columns)
    figure1=figure(x_range=sellers,height=250,title=column_values,plot_width=1300,plot_height=1200,
              tools='reset,box_zoom',toolbar_location='below',tooltips="$name @"+column_index+":@$name")
    figure1.xgrid.grid_line_color=None

    figure1.xaxis.axis_label=column_index
    figure1.yaxis.axis_label='$'+column_values
    figure1.xaxis.major_label_text_font_size='50px'
    figure1.xgrid.grid_line_color=None
    figure1.ygrid.grid_line_alpha=0.7
    figure1.yaxis.major_label_text_font_size='50px'

    figure1.toolbar.autohide=True
    figure1.y_range.start = 0
    figure1.x_range.range_padding = 0.1
    figure1.xaxis.major_label_orientation = 1
    figure1.xgrid.grid_line_color = None
    figure1.vbar_stack(columns[1:-1],x=column_index, width=0.9, color=['#922B21','#B03A2E','#76448A','#6C3483',
                                                              '#1F618D','#2874A6','#117864','#0B5345',
                                                             '#196F3D','#7D6608','#6E2C00','#F5CBA7',
                                                             '#85C1E9'],line_color="#17202A", 
                    source=TABLA_TOTAL_SORT,legend_label=columns[1:-1])
    return figure1


    

In [15]:
barra_stackeada1=barra_stackeada(DATA_FRAME,'sellers','PRODUCTNAME','profit')

In [16]:
show(barra_stackeada1)